Let's start by creating a single dataframe for all the seasons I have available

In [65]:
import pandas as pd
import numpy as np
import sys

In [66]:
#e0_1112 = pd.read_csv('../data/England/E0_1112.csv')
e0_1213 = pd.read_csv('../data/England/E0_1213.csv')
e0_1314 = pd.read_csv('../data/England/E0_1314.csv')
e0_1415 = pd.read_csv('../data/England/E0_1415.csv')

#Let's drop some columns so that the dataframes match exactly and it doesn't screw the concat
e0_1213.drop(['GBH', 'GBD', 'GBA','BSH', 'BSD', 'BSA'], axis=1, inplace=True)
e0_1314.drop('Referee', axis=1, inplace=True)
e0_1415.drop('Referee', axis=1, inplace=True)

frames = [e0_1213, e0_1314, e0_1415]

df0 = pd.concat(frames, ignore_index=True)
df0.head()
#df0.shape

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA
0,E0,18/08/12,Arsenal,Sunderland,0,0,D,0,0,D,...,1.83,1.76,2.15,2.05,23,-1.25,2.02,1.96,1.96,1.91
1,E0,18/08/12,Fulham,Norwich,5,0,H,2,0,H,...,1.95,1.89,2.01,1.92,21,-0.50,1.83,1.80,2.14,2.09
2,E0,18/08/12,Newcastle,Tottenham,2,1,H,0,0,D,...,2.00,1.88,2.01,1.92,23,0.00,1.93,1.88,2.03,1.97
3,E0,18/08/12,QPR,Swansea,0,5,A,0,1,A,...,2.18,2.09,1.81,1.73,22,-0.25,1.80,1.75,2.21,2.13
4,E0,18/08/12,Reading,Stoke,1,1,D,0,1,A,...,2.28,2.18,1.76,1.67,24,-0.25,2.07,2.01,1.91,1.86


Create new DataFrame with the columns I'm interested in-- Team names and full-time results for now

In [67]:
df1 = df0.ix[:,2:7].copy()
df1.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,Arsenal,Sunderland,0,0,D
1,Fulham,Norwich,5,0,H
2,Newcastle,Tottenham,2,1,H
3,QPR,Swansea,0,5,A
4,Reading,Stoke,1,1,D


Create new columns to include: "lookback" Home/Away goals scored, goals conceded, and number of points

In [68]:
df1['LB_HGF'] = 0
df1['LB_AGF'] = 0
df1['LB_HGA'] = 0
df1['LB_AGA'] = 0
df1['LB_HP'] = 0
df1['LB_AP'] = 0
df1['FLAG'] = 0  #quality flag, i.e., enough matches found to compute lookback results
df1.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,LB_HGF,LB_AGF,LB_HGA,LB_AGA,LB_HP,LB_AP,FLAG
0,Arsenal,Sunderland,0,0,D,0,0,0,0,0,0,0
1,Fulham,Norwich,5,0,H,0,0,0,0,0,0,0
2,Newcastle,Tottenham,2,1,H,0,0,0,0,0,0,0
3,QPR,Swansea,0,5,A,0,0,0,0,0,0,0
4,Reading,Stoke,1,1,D,0,0,0,0,0,0,0


Define some parameters-- for now, only the lookback_matches seems relevant

In [69]:
params = {
    'lookback': 5  #number of matches to look back at
}

#define a couple of dicts to translate results keys to points
hpoints = {'H': 3, 'D': 1, 'A': 0}
apoints = {'H': 0, 'D': 1, 'A': 3}

Suggested logic to get goals/points in X previous matches:

- iterate over the list
- get name of HomeTeam
  - start iterating back
  - look for matching name in HomeTeam
  - otherwise, in AwayTeam
  - if match is found, count goals/points
  - count a match, X-1 to go....
- get name of AwayTeam, repeat prev steps

In [70]:
for index, row in df1.iterrows():
    hometeam = row['HomeTeam']
    awayteam = row['AwayTeam']
    
    #HomeTeam
    lb_hgames = 0  #number of previous games found for home team
    lb_hgf = 0  #cumulative number of goals scored
    lb_hga = 0  #cumulative number of goals conceded
    lb_hpoints = 0  #cumulative number of points
    for j in reversed(xrange(index)):
        if(df1.loc[j,'HomeTeam']==hometeam):
            lb_hgf = lb_hgf + df1.loc[j,'FTHG']
            lb_hga = lb_hga + df1.loc[j,'FTAG']
            lb_hpoints = lb_hpoints + hpoints[df1.loc[j,'FTR']]
            lb_hgames+=1
        elif(df1.loc[j,'AwayTeam']==hometeam):
            lb_hgf = lb_hgf + df1.loc[j,'FTAG']
            lb_hga = lb_hga + df1.loc[j,'FTHG']
            lb_hpoints = lb_hpoints + apoints[df1.loc[j,'FTR']]
            lb_hgames+=1
        if(lb_hgames==params['lookback']):
            break
    df1.loc[index,'LB_HGF'] = lb_hgf
    df1.loc[index,'LB_HGA'] = lb_hga
    df1.loc[index,'LB_HP'] = lb_hpoints
    
    #AwayTeam
    lb_agames, lb_agf, lb_aga, lb_apoints = 0, 0, 0, 0  #reinitialize lookback vars
    for j in reversed(xrange(index)):
        if(df1.loc[j,'HomeTeam']==awayteam):
            lb_agf = lb_agf + df1.loc[j,'FTHG']
            lb_aga = lb_aga + df1.loc[j,'FTAG']
            lb_apoints = lb_apoints + hpoints[df1.loc[j,'FTR']]
            lb_agames+=1
        elif(df1.loc[j,'AwayTeam']==awayteam):
            lb_agf = lb_agf + df1.loc[j,'FTAG']
            lb_aga = lb_aga + df1.loc[j,'FTHG']
            lb_apoints = lb_apoints + apoints[df1.loc[j,'FTR']]
            lb_agames+=1
        if(lb_agames==params['lookback']):
            break
    df1.loc[index,'LB_AGF'] = lb_agf
    df1.loc[index,'LB_AGA'] = lb_aga
    df1.loc[index,'LB_AP'] = lb_apoints
    
    if (lb_hgames==params['lookback']) & (lb_agames==params['lookback']):
        df1.loc[index,'FLAG'] = 1

Drop rows with not enough lookback games (FLAG = 0)

In [71]:
df1 = df1[df1.FLAG != 0]
df1.drop('FLAG', axis=1, inplace=True)
df1.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,LB_HGF,LB_AGF,LB_HGA,LB_AGA,LB_HP,LB_AP
49,Arsenal,Chelsea,1,2,A,9,9,2,2,9,13
50,Everton,Southampton,3,1,H,9,9,5,15,10,3
51,Fulham,Man City,1,2,A,12,10,7,7,9,9
52,Man United,Tottenham,2,3,A,12,8,6,6,12,8
53,Norwich,Liverpool,2,5,A,2,4,8,10,3,2


In [63]:
df1.to_csv('../data/England/E0_1215_feats.csv', index=False)

In [64]:
df1.shape

(1065, 12)

In [118]:
df2 = df1[(df1['HomeTeam'].isin(['Arsenal'])) | (df1['AwayTeam'].isin(['Arsenal']))]
#df2.reset_index(drop=True, inplace=True)
#df2.loc[0,'FTHG']=9
df2.head()
#for index, row in df2.iterrows():
#  print index, row['HomeTeam'], row['AwayTeam']
#for i, row in enumerate(df2.values):
#    print row[1]

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,LB_HGF,LB_AGF,LB_HGA,LB_AGA,LB_HP,LB_AP,FLAG
0,Arsenal,Aston Villa,1,3,A,0,0,0,0,0,0,0
13,Fulham,Arsenal,1,3,A,1,1,0,3,3,0,0
27,Arsenal,Tottenham,1,0,H,4,2,4,0,3,6,1
36,Sunderland,Arsenal,1,3,A,2,4,4,1,1,6,1
46,Arsenal,Stoke,3,1,H,4,1,1,0,6,4,1


In [81]:
team = "Arsenal"
i = 46
df2.loc[i,:].values
lookback = 2
lb_matches = 0
lb_goals = 0

for j in reversed(xrange(i)):
    if(df1.loc[j,"HomeTeam"]==team):
        lb_goals = lb_goals + df1.loc[j,"FTHG"]
        lb_matches+=1
        #print "goals scored by arsenal: %d" %df1.loc[j,"FTHG"]
    elif(df1.loc[j,"AwayTeam"]==team):
        lb_goals = lb_goals + df1.loc[j,"FTAG"]
        lb_matches+=1
        #print "goals scored by arsenal: %d" %df1.loc[j,"FTAG"]
    if(lb_matches==lookback):
        break
    
        #print df1.loc[j,:]
print "Number of goals scored by %s in the last %d matches: %d" %(team, lookback, lb_goals)    

Number of goals scored by Arsenal in the last 2 matches: 4


In [37]:
teams = df1.loc[:,'HomeTeam'].unique()
for team in teams:
    print team

Arsenal
Liverpool
Norwich
Sunderland
Swansea
West Brom
West Ham
Chelsea
Crystal Palace
Man City
Aston Villa
Everton
Fulham
Hull
Newcastle
Southampton
Stoke
Cardiff
Tottenham
Man United
Leicester
QPR
Burnley
